## ResNet Models

#### Imports

In [2]:
# Run if using tensorflow2.0+
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Add, Activation, InputLayer
from tensorflow.keras import Input
from tensorflow.keras.optimizers import SGD, Adam

#### Read Data

In [3]:
df_train = pd.read_csv('../Data/train.csv')
df_train['open_channels'].value_counts(normalize=True)

0     0.248030
1     0.197173
3     0.133722
2     0.110785
4     0.080682
5     0.055575
7     0.053003
8     0.049037
6     0.037622
9     0.027224
10    0.007147
Name: open_channels, dtype: float64

Method that cuts up the data into batches

In [4]:
def get_data(batch_num='all', zeros = True):
    #create training data per batch
    batch_size = 500000
    if batch_num == 'all':
        beg = 0
        end = 499999999
        print('Training on all data')
    else:
        batch = batch_num
        beg = (batch-1)*batch_size 
        end = batch*batch_size - 1
    df_batch = df_train[beg:end]
    
    # filter out the zero open_channels cases
    if not zeros:
        df_batch = df_batch.drop(df_batch[df_batch['open_channels']==0].index) #optional, added because model was always predicting 0
        
    # randomize 
    signal = np.array(df_batch.signal)
    open_channels = np.array(df_batch.open_channels)
    x_train, x_test, y_train, y_test = train_test_split(signal,open_channels,test_size=0.25)
    x_train = np.reshape(x_train, (-1,1))
    y_train = np.reshape(y_train, (-1,1))
    x_test = np.reshape(x_test, (-1,1))
    y_test = np.reshape(y_test, (-1,1))
    
    # categorize outputs
    enc = OneHotEncoder()
    enc.fit_transform(y_train)
    enc.fit_transform(y_test)
    
    return x_train, x_test, y_train, y_test

#### Network

Network definition has options for dense layer size, the amount of dropout in each residual block, and how many residual blocks to use

In [5]:
def resnet(layer_size=64, dropout=0, depth=1):
    #model params
    layer = layer_size
    dropout = dropout
    
    #create model
    inputs = Input(shape=(1,))

    if depth > 0:
        k = Dense(layer, activation='relu')(inputs)
        k = Dense(layer, activation='relu')(k)
        k = Dropout(dropout)(k)
        block1 = Add()([k, inputs])
        kl = Dense(20, activation='relu')(block1)
        
        if depth > 1:
            k = Dense(layer, activation='relu')(block1)
            k = Dense(layer, activation='relu')(k)
            k = Dropout(dropout)(k)
            block2 = Add()([k, block1])
            kl = Dense(20, activation='relu')(block2)

            if depth > 2:
                k = Dense(layer, activation='relu')(block2)
                k = Dense(layer, activation='relu')(k)
                k = Dropout(dropout)(k)
                block3 = Add()([k, block2])
                kl = Dense(20, activation='relu')(block3)

                if depth > 3:
                    k = Dense(layer, activation='relu')(block3)
                    k = Dense(layer, activation='relu')(k)
                    k = Dropout(dropout)(k)
                    block4 = Add()([k, block3])
                    kl = Dense(20, activation='relu')(block4)
                    
                    if depth > 4:
                        k = Dense(layer, activation='relu')(block4)
                        k = Dense(layer, activation='relu')(k)
                        k = Dropout(dropout)(k)
                        block5 = Add()([k, block4])
                        kl = Dense(20, activation='relu')(block5)
                        
                        if depth > 5:
                            k = Dense(layer, activation='relu')(block5)
                            k = Dense(layer, activation='relu')(k)
                            k = Dropout(dropout)(k)
                            block6 = Add()([k, block5])
                            kl = Dense(20, activation='relu')(block6)
                        
                            if depth > 6:
                                k = Dense(layer, activation='relu')(block6)
                                k = Dense(layer, activation='relu')(k)
                                k = Dropout(dropout)(k)
                                block7 = Add()([k, block6])
                                kl = Dense(20, activation='relu')(block7)
                                
                                if depth > 7:
                                    k = Dense(layer, activation='relu')(block7)
                                    k = Dense(layer, activation='relu')(k)
                                    k = Dropout(dropout)(k)
                                    block8 = Add()([k, block7])
                                    kl = Dense(20, activation='relu')(block8)

    outputs = Dense(units=11, activation='softmax')(kl)
    
    model = Model(inputs,outputs)
    return model

#### Training

In [50]:
x_train,x_test,y_train,y_test = get_data(1,zeros=True)
model = resnet(layer_size=256, dropout=0.1, depth=8)
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test), batch_size=100, verbose=1)

Train on 374999 samples, validate on 125000 samples
Epoch 1/5
374999/374999 [==============================] - 42s 111us/sample - loss: 0.0890 - accuracy: 0.9682 - val_loss: 0.0838 - val_accuracy: 0.9677
Epoch 2/5
374999/374999 [==============================] - 46s 122us/sample - loss: 0.0716 - accuracy: 0.9691 - val_loss: 0.0479 - val_accuracy: 0.9749
Epoch 3/5
374999/374999 [==============================] - 45s 121us/sample - loss: 0.0385 - accuracy: 0.9865 - val_loss: 0.0079 - val_accuracy: 0.9977
Epoch 4/5
374999/374999 [==============================] - 45s 121us/sample - loss: 0.0137 - accuracy: 0.9958 - val_loss: 0.0163 - val_accuracy: 0.9948
Epoch 5/5
374999/374999 [==============================] - 46s 121us/sample - loss: 0.0094 - accuracy: 0.9969 - val_loss: 0.0300 - val_accuracy: 0.9892


Full data model

In [27]:
x_train,x_test,y_train,y_test = get_data(zeros=True)
modelf = resnet(layer_size=20, dropout=0.1, depth=3)
modelf.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
modelf.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test), batch_size=80, verbose=1)

Training on all data
Train on 3750000 samples, validate on 1250000 samples
Epoch 1/10
3750000/3750000 [==============================] - 61s 16us/sample - loss: 1.2219 - accuracy: 0.5453 - val_loss: 1.1526 - val_accuracy: 0.5773
Epoch 2/10
3750000/3750000 [==============================] - 64s 17us/sample - loss: 1.1500 - accuracy: 0.5732 - val_loss: 1.1376 - val_accuracy: 0.5779
Epoch 3/10
3750000/3750000 [==============================] - 66s 18us/sample - loss: 1.1412 - accuracy: 0.5744 - val_loss: 1.1401 - val_accuracy: 0.5712
Epoch 4/10
3750000/3750000 [==============================] - 67s 18us/sample - loss: 1.1381 - accuracy: 0.5750 - val_loss: 1.1398 - val_accuracy: 0.5744
Epoch 5/10
3750000/3750000 [==============================] - 69s 19us/sample - loss: 1.1363 - accuracy: 0.5757 - val_loss: 1.1366 - val_accuracy: 0.5740
Epoch 6/10
3750000/3750000 [==============================] - 68s 18us/sample - loss: 1.1351 - accuracy: 0.5760 - val_loss: 1.1353 - val_accuracy: 0.5813
E

In [28]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
pred = modelf.predict(x)

In [29]:
correct = 0
incorrect = 0

for i in range(len(y)):
    if np.argmax(pred[i]) == y[i]:
        correct += 1
    else:
        incorrect += 1

print('Accuracy:',correct/(correct+incorrect))

Accuracy: 0.5895556


Flat data model

In [18]:
# Create model for flat signal type (batches 1-6)
x_tr1,x_t1,y_tr1,y_t1 = get_data(1)
x_tr2,x_t2,y_tr2,y_t2 = get_data(2)
x_tr3,x_t3,y_tr3,y_t3 = get_data(3)
x_tr4,x_t4,y_tr4,y_t4 = get_data(4)
x_tr5,x_t5,y_tr5,y_t5 = get_data(5)
x_tr6,x_t6,y_tr6,y_t6 = get_data(6)
x_train = np.concatenate((x_tr1,x_tr2,x_tr3,x_tr4,x_tr5,x_tr6))
x_test = np.concatenate((x_t1,x_t2,x_t3,x_t4,x_t5,x_t6))
y_train = np.concatenate((y_tr1,y_tr2,y_tr3,y_tr4,y_tr5,y_tr6))
y_test = np.concatenate((y_t1,y_t2,y_t3,y_t4,y_t5,y_t6))

In [8]:
model1 = resnet(layer_size=20, dropout=0.1, depth=2) #175,4
model1.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model1.fit(x_train, y_train, epochs=6, validation_data=(x_test,y_test), batch_size=80, verbose=1)

Train on 2249994 samples, validate on 750000 samples
Epoch 1/6
2249994/2249994 [==============================] - 37s 16us/sample - loss: 0.4956 - accuracy: 0.8280 - val_loss: 0.3937 - val_accuracy: 0.8565
Epoch 2/6
2249994/2249994 [==============================] - 38s 17us/sample - loss: 0.3976 - accuracy: 0.8554 - val_loss: 0.3895 - val_accuracy: 0.8574
Epoch 3/6
2249994/2249994 [==============================] - 39s 17us/sample - loss: 0.3864 - accuracy: 0.8591 - val_loss: 0.3776 - val_accuracy: 0.8633
Epoch 4/6
2249994/2249994 [==============================] - 40s 18us/sample - loss: 0.3830 - accuracy: 0.8599 - val_loss: 0.3840 - val_accuracy: 0.8609
Epoch 5/6
2249994/2249994 [==============================] - 40s 18us/sample - loss: 0.3812 - accuracy: 0.8607 - val_loss: 0.3751 - val_accuracy: 0.8617
Epoch 6/6
2249994/2249994 [==============================] - 41s 18us/sample - loss: 0.3801 - accuracy: 0.8609 - val_loss: 0.3775 - val_accuracy: 0.8626


In [19]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
pred = model1.predict(x)

In [20]:
correct = 0
incorrect = 0

for i in range(len(y)):#range(len(x_test)):
    if np.argmax(pred[i]) == y[i]:
        correct += 1
    else:
        incorrect += 1

print('Accuracy:',correct/(correct+incorrect))

Accuracy: 0.8626407252814505


In [71]:
model1.save('../Models/resnet_flat_85.h5')

Parabolic data model

In [24]:
# Create model for parabolic signal type (batches 2,7-10)
x_tr2,x_t2,y_tr2,y_t2 = get_data(2)
x_tr7,x_t7,y_tr7,y_t7 = get_data(7)
x_tr8,x_t8,y_tr8,y_t8 = get_data(8)
x_tr9,x_t9,y_tr9,y_t9 = get_data(9)
x_tr10,x_t10,y_tr10,y_t10 = get_data(10)
x_train = np.concatenate((x_tr2,x_tr7,x_tr8,x_tr9,x_tr10))
x_test = np.concatenate((x_t2,x_t7,x_t8,x_t9,x_t10))
y_train = np.concatenate((y_tr2,y_tr7,y_tr8,y_tr9,y_tr10))
y_test = np.concatenate((y_t2,y_t7,y_t8,y_t9,y_t10))

In [9]:
model = resnet(layer_size=30, dropout=0.1, depth=2) #300,2
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=6, validation_data=(x_test,y_test), batch_size=80, verbose=1)

Train on 1874995 samples, validate on 625000 samples
Epoch 1/6
1874995/1874995 [==============================] - 32s 17us/sample - loss: 1.3607 - accuracy: 0.4774 - val_loss: 1.2941 - val_accuracy: 0.5087
Epoch 2/6
1874995/1874995 [==============================] - 34s 18us/sample - loss: 1.2926 - accuracy: 0.5032 - val_loss: 1.3212 - val_accuracy: 0.4759
Epoch 3/6
1874995/1874995 [==============================] - 35s 19us/sample - loss: 1.2770 - accuracy: 0.5100 - val_loss: 1.2652 - val_accuracy: 0.5132
Epoch 4/6
1874995/1874995 [==============================] - 35s 19us/sample - loss: 1.2694 - accuracy: 0.5129 - val_loss: 1.2775 - val_accuracy: 0.5051
Epoch 5/6
1874995/1874995 [==============================] - 35s 19us/sample - loss: 1.2628 - accuracy: 0.5152 - val_loss: 1.2748 - val_accuracy: 0.5115
Epoch 6/6
1874995/1874995 [==============================] - 36s 19us/sample - loss: 1.2580 - accuracy: 0.5169 - val_loss: 1.2471 - val_accuracy: 0.5238


In [25]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
pred = model.predict(x)

In [26]:
correct = 0
incorrect = 0

for i in range(len(y)):#range(len(x_test)):
    if np.argmax(pred[i]) == y[i]:
        correct += 1
    else:
        incorrect += 1

print('Accuracy:',correct/(correct+incorrect))

Accuracy: 0.523508647017294


In [72]:
model.save('../Models/resnet_curvy_49.h5')